In [9]:
import os
ipv6_ip_block = '2001:470:8863:1aba'
def extract_domains(pcap_file:str)->tuple[set[str], set[str], dict[str, str], dict[str, str]]:
    # unique_slds = set()
    domain_list = set()
    ip_domain_map = {}
    # sld_ip_map = {}
    
    # Extract domain names from DNS queries
    dns_output = os.popen(f"tshark -r {pcap_file} -Y \"dns.flags.response==1 && not mdns\" -T fields -e dns.qry.name -e dns.qry.type -e dns.a -e dns.aaaa").read()
    hosts = dns_output.splitlines()
    for line in hosts:
        line = line.split("\t")
        if len(line) < 4:
            continue
        # Local traffic filtering
        if line[0].startswith('192.168.') or line[0].startswith(ipv6_ip_block) or 'in-addr.arpa' in line[0] or '.local' in line[0]:
            continue
        print(line)
        domain = line[0].lower()
        dns_type = line[1]
        ips = line[3].split(",") if dns_type == '28' else line[2].split(",")
        if domain and domain[-1] == '.':
            domain = domain[:-1]
        # sld = extract_sld(domain)
        # if not sld:
        #     continue
        print(ips)
        domain_list.add(domain)
        # unique_slds.add(sld)
        for ip in ips:
            if len(ip) == 0:
                print(f"Empty IP for domain: {domain}")
                continue
            ip_domain_map[ip] = domain
            # sld_ip_map[ip] = sld
    
    # Extract domain names from TLS handshake
    tls_output = os.popen(f"tshark -r {pcap_file} -Y \"tls.handshake.extensions_server_name\" -T fields -e tls.handshake.extensions_server_name -e ip.dst").read()
    tls_hosts = tls_output.splitlines()
    for line in tls_hosts:
        line = line.split("\t")
        if len(line) < 2:
            continue
        if line[1].startswith('192.168.') or 'in-addr.arpa' in line[1] or '.local' in line[1]:
            continue
        domain = line[0].lower()
        if domain and domain[-1] == '.':
            domain = domain[:-1]
        ip = line[1]
        # print(line)
        # sld = extract_sld(domain)
        # unique_slds.add(sld)
        domain_list.add(domain)
        # for ip in ips:
        ip_domain_map[ip] = domain
            # sld_ip_map[ip] = sld
        
        
    
    # return domain_list, ip_domain_map

In [10]:
extract_domains('/net/data/iot-longitudinal/datasets/2019-datasets/idle-us-copy/google-home-mini/2019-04-27_16.49.03_192.168.10.225.pcap')

['www.google.com', '1', '172.217.10.36', '']
['172.217.10.36']
['www.google.com', '28', '', '2607:f8b0:4006:803::2004']
['2607:f8b0:4006:803::2004']
['connectivitycheck.gstatic.com', '1', '172.217.12.131', '']
['172.217.12.131']
['www.google.com', '1', '172.217.10.36', '']
['172.217.10.36']
['www.google.com', '28', '', '2607:f8b0:4006:803::2004']
['2607:f8b0:4006:803::2004']
['connectivitycheck.gstatic.com', '1', '172.217.11.3', '']
['172.217.11.3']
['clients4.google.com', '1', '172.217.12.174', '']
['172.217.12.174']
['www.google.com', '28', '', '2607:f8b0:4006:818::2004']
['2607:f8b0:4006:818::2004']
['www.google.com', '1', '172.217.10.36', '']
['172.217.10.36']
['connectivitycheck.gstatic.com', '1', '172.217.11.3', '']
['172.217.11.3']
['clients4.google.com', '1', '172.217.11.46', '']
['172.217.11.46']
['www.google.com', '28', '', '2607:f8b0:4006:803::2004']
['2607:f8b0:4006:803::2004']
['www.google.com', '1', '172.217.10.36', '']
['172.217.10.36']
['connectivitycheck.gstatic.com', 

In [4]:
import ipaddress
def is_local_address(ip_str):
    LOCAL_IPS = ['129.10.227.248', '129.10.227.207']
    try:
        if ip_str == "::" or ip_str == "::1":
            return True
        if is_ipv6(ip_str):
            print('IPv6', ip_str)
            ip = ipaddress.IPv6Address(ip_str)
            return ip.is_multicast or ip.is_link_local or (
                ip.is_private and check_in_network("fc00::/7", ip)
                ) or ip.is_unspecified
        else:
            print('IPv4', ip_str)
            ip = ipaddress.ip_address(ip_str)
            return ip.is_private or ip.is_multicast or ip == ipaddress.IPv4Address("255.255.255.255") or ip_str in LOCAL_IPS
    except Exception as e:
        print(ip_str, e)
        return False
def check_in_network(network_prefix, ip):
    if ip is None or network_prefix is None:
        return False
    network = ipaddress.IPv6Network(network_prefix, strict=False)
    return ip in network

def is_ipv6(address:str) -> bool:
    try:
        ip = ipaddress.ip_address(address)
        if isinstance(ip, ipaddress.IPv6Address):
            # print("{} is an IPv6 address".format(address))
            return True
        else:
            return False
    except ValueError:
        return False
is_local_address('192.168.10.214')

IPv4 192.168.10.214


True